In [1]:
# import common packages
from collections import OrderedDict
import itertools
import logging

import numpy as np

from qiskit import Aer

from qiskit_aqua import Operator, set_aqua_logging, QuantumInstance
from qiskit_aqua.algorithms.adaptive import VQE
from qiskit_aqua.algorithms.classical import ExactEigensolver
from qiskit_aqua.algorithms.components.optimizers import COBYLA

from qiskit_aqua_chemistry.drivers import ConfigurationManager
from qiskit_aqua_chemistry.core import Hamiltonian
from qiskit_aqua_chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit_aqua_chemistry.aqua_extensions.components.initial_states import HartreeFock

# set_aqua_logging(logging.INFO)

In [2]:
# using driver to get fermionic Hamiltonian
cfg_mgr = ConfigurationManager()
pyscf_cfg = OrderedDict([('atom', 'Li .0 .0 .0; H .0 .0 1.6'), 
                         ('unit', 'Angstrom'), ('charge', 0), 
                         ('spin', 0), ('basis', 'sto3g')])
section = {}
section['properties'] = pyscf_cfg
driver = cfg_mgr.get_driver_instance('PYSCF')
molecule = driver.run(section)

In [3]:
core = Hamiltonian(transformation='full', qubit_mapping='parity', 
                   two_qubit_reduction=True, freeze_core=True)
algo_input = core.run(molecule)
qubit_op = algo_input.qubit_op

print("Originally requires {} qubits".format(qubit_op.num_qubits))
print(qubit_op)

Originally requires 8 qubits
Representation: paulis, qubits: 8, size: 276


Find the symmetries of the qubit operator

In [4]:
[symmetries, sq_paulis, cliffords, sq_list] = qubit_op.find_Z2_symmetries()
print('Z2 symmetries found:')
for symm in symmetries:
    print(symm.to_label())
print('single qubit operators found:')
for sq in sq_paulis:
    print(sq.to_label())
print('cliffords found:')
for clifford in cliffords:
    print(clifford.print_operators())
print('single-qubit list: {}'.format(sq_list))

Z2 symmetries found:
ZIZIZIZI
ZZIIZZII
single qubit operators found:
IIIIIIXI
IIIIIXII
cliffords found:
ZIZIZIZI	0.7071067811865475
IIIIIIXI	0.7071067811865475

ZZIIZZII	0.7071067811865475
IIIIIXII	0.7071067811865475

single-qubit list: [1, 2]


Use the found symmetries, single qubit operators, and cliffords to taper qubits from the original qubit operator. For each Z2 symmetry one can taper one qubit. However, different tapered operators can be built, corresponding to different symmetry sectors. 

In [5]:
tapered_ops = []
for coeff in itertools.product([1, -1], repeat=len(sq_list)):
    tapered_op = Operator.qubit_tapering(qubit_op, cliffords, sq_list, list(coeff))
    tapered_ops.append((list(coeff), tapered_op))
    print("Number of qubits of tapered qubit operator: {}".format(tapered_op.num_qubits))

Number of qubits of tapered qubit operator: 6
Number of qubits of tapered qubit operator: 6
Number of qubits of tapered qubit operator: 6
Number of qubits of tapered qubit operator: 6


The user has to specify the symmetry sector he is interested in. Since we are interested in finding the ground state here, let us get the original ground state energy as a reference.

In [6]:
ee = ExactEigensolver(qubit_op, k=1)
result = core.process_algorithm_result(ee.run())
for line in result[0]:
    print(line)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.874303870396
  - computed part:      -1.078084301625
  - frozen energy part: -7.796219568771
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.992207270475
> Total ground state energy (Hartree): -7.882096599921


Now, let us iterate through all tapered qubit operators to find out the one whose ground state energy matches the original (un-tapered) one.

In [7]:
smallest_eig_value = 99999999999999
smallest_idx = -1
for idx in range(len(tapered_ops)):
    ee = ExactEigensolver(tapered_ops[idx][1], k=1)
    curr_value = ee.run()['energy']
    if curr_value < smallest_eig_value:
        smallest_eig_value = curr_value
        smallest_idx = idx
    print("Lowest eigenvalue of the {}-th tapered operator (computed part) is {:.12f}".format(idx, curr_value))
    
the_tapered_op = tapered_ops[smallest_idx][1]
the_coeff = tapered_ops[smallest_idx][0]
print("The {}-th tapered operator matches original ground state energy, with corresponding symmetry sector of {}".format(smallest_idx, the_coeff))

Lowest eigenvalue of the 0-th tapered operator (computed part) is -1.078084301625
Lowest eigenvalue of the 1-th tapered operator (computed part) is -0.509523578167
Lowest eigenvalue of the 2-th tapered operator (computed part) is -0.912078232998
Lowest eigenvalue of the 3-th tapered operator (computed part) is -0.912078232998
The 0-th tapered operator matches original ground state energy, with corresponding symmetry sector of [1, 1]


Alternatively, one can run multiple VQE instances to find the lowest eigenvalue sector. 
Here we just validate that `the_tapered_op` reach the smallest eigenvalue in one VQE execution with the UCCSD variational form, modified to take into account of the tapered symmetries.

In [8]:
# setup initial state
init_state = HartreeFock(num_qubits=the_tapered_op.num_qubits, num_orbitals=core._molecule_info['num_orbitals'],
                    qubit_mapping=core._qubit_mapping, two_qubit_reduction=core._two_qubit_reduction,
                    num_particles=core._molecule_info['num_particles'], sq_list=sq_list)

# setup variationl form
var_form = UCCSD(num_qubits=the_tapered_op.num_qubits, depth=1,
                   num_orbitals=core._molecule_info['num_orbitals'], 
                   num_particles=core._molecule_info['num_particles'],
                   active_occupied=None, active_unoccupied=None, initial_state=init_state,
                   qubit_mapping=core._qubit_mapping, two_qubit_reduction=core._two_qubit_reduction, 
                   num_time_slices=1,
                   cliffords=cliffords, sq_list=sq_list, tapering_values=the_coeff, symmetries=symmetries)

# setup optimizer
optimizer = COBYLA()
optimizer.set_options(maxiter=1000)

# set vqe
algo = VQE(the_tapered_op, var_form, optimizer, 'matrix')

# setup backend
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=backend)

In [9]:
algo_result = algo.run(quantum_instance)

In [10]:
result = core.process_algorithm_result(algo_result)
for line in result[0]:
    print(line)

print("The parameters for UCCSD are:\n{}".format(algo_result['opt_params']))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.874303856889
  - computed part:      -1.078084288118
  - frozen energy part: -7.796219568771
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.992207270475
> Total ground state energy (Hartree): -7.882096586414
The parameters for UCCSD are:
[ 0.03815735  0.00366554  0.03827111  0.00369737 -0.03604811  0.0594364
 -0.02741369 -0.02735108  0.05956488 -0.11497243]
